In [1]:
import os

%pwd

'd:\\work\\cifar100classifier-deep-learning\\research'

In [2]:
os.chdir('../')

%pwd

'd:\\work\\cifar100classifier-deep-learning'

In [3]:
import sys

sys.path.append('src')

%pwd

'd:\\work\\cifar100classifier-deep-learning'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
from CNNClassifier.constants import *
from CNNClassifier.utils.utilities import read_yaml, create_directory

In [5]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directory([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directory([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [7]:
import time
from livelossplot.inputs.keras import PlotLossesCallback

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True,
            verbose=1
        )
        return checkpoint
    
    @property
    def _create_es_callbacks(self):
        early_stop = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            patience=5, 
            restore_best_weights=True, 
            mode='min'
        )
        return early_stop
    
    @property
    def _create_plot_loss(self):
        return PlotLossesCallback()
    
    def callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks,
            self._create_es_callbacks,
            self._create_plot_loss,
        ]

In [8]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2023-10-20 14:34:05,282: INFO: utilities]: yaml file: configs\config.yaml loaded successfully
[2023-10-20 14:34:05,287: INFO: utilities]: yaml file: params.yaml loaded successfully
[2023-10-20 14:34:05,290: INFO: utilities]: create directory at: artifacts
[2023-10-20 14:34:05,316: INFO: utilities]: create directory at: artifacts\prepare_callbacks\checkpoint_dir
[2023-10-20 14:34:05,319: INFO: utilities]: create directory at: artifacts\prepare_callbacks\tensorboard_log_dir
